In [1]:
from hidromet import config
from hidromet import modelos
from hidromet import utils

import geopandas as gpd
from pathlib import Path
import pandas as pd
import json

In [2]:
gdf = gpd.read_file(config.dir_contorno)
crs = gdf.crs
bacia = gdf.iloc[0]
nome_bacia = bacia.bacia

### Remover repetidos

In [3]:
arquivo_coords_inmet = config.dir_prec_inmet.joinpath(f"coords_{nome_bacia}.json")
arquivo_serie_inmet = config.dir_prec_inmet.joinpath(f"{nome_bacia}.csv")

json_inmet = utils.carregar_json(arquivo_coords_inmet)
postos_ok = json_inmet['postos_ok']
df_inmet = pd.DataFrame.from_dict(postos_ok).round(2)

series_bacia_inmet = pd.read_csv(arquivo_serie_inmet, index_col=0)
postos_inmet = df_inmet.codigo
serie_inmet = series_bacia_inmet[postos_inmet]

In [4]:
arquivo_serie_ana = config.dir_prec_ana.joinpath(f"{nome_bacia}.csv")
arquivo_coords_ana = config.dir_prec_ana.joinpath(f"coords_{nome_bacia}.json")

json_ana = utils.carregar_json(arquivo_coords_ana)
postos_ok = json_ana['postos_ok']
df_ana = pd.DataFrame.from_dict(postos_ok).round(2)

series_bacia_ana = pd.read_csv(arquivo_serie_ana, index_col=0)
postos_ana = df_ana.codigo
serie_ana = series_bacia_ana[postos_ana]

In [5]:
postos_duplicados = df_ana.merge(df_inmet, left_on=["latitude", "longitude"], right_on=["latitude", "longitude"], suffixes=("_ana", "_inmet"))

In [6]:
if not postos_duplicados.empty:

    for ix in postos_duplicados.index:

        posto_duplicados_ana = postos_duplicados.loc[ix, "codigo_ana"]
        posto_duplicados_inmet = postos_duplicados.loc[ix, "codigo_inmet"]

        serie_duplicada_ana = series_bacia_ana[posto_duplicados_ana].dropna()
        serie_duplicada_inmet = series_bacia_inmet[posto_duplicados_inmet].dropna()

        if len(serie_duplicada_ana) > len(posto_duplicados_inmet):
            serie_inmet.drop([posto_duplicados_inmet], axis=1)
            df_inmet = df_inmet[df_inmet.codigo != posto_duplicados_inmet]
        else:
            serie_ana.drop([posto_duplicados_ana], axis=1)
            df_ana = df_ana[df_ana.codigo != posto_duplicados_ana]

log_ana = [modelos.Posto(**df_ana.loc[ix]).dict() for ix in df_ana.index]
log_inmet = [modelos.Posto(**df_inmet.loc[ix]).dict() for ix in df_inmet.index]

logs = [*log_ana, *log_inmet]
arquivo_json = Path(config.dir_prec_concat, f'coords_{nome_bacia}.json')
with open(arquivo_json, 'w') as f:
    json.dump(logs, f)

df = pd.concat([serie_ana, serie_inmet], axis=1)
df.to_csv(f"{config.dir_prec_concat}/{nome_bacia}.csv")
        